In [1]:
import torch.nn as nn
import torch
import torchaudio
import matplotlib.pyplot as plt
import csv
import numpy as np
import sys
import os
import datetime
sys.path.append(os.path.dirname(os.path.dirname(sys.path[0])))
from utilities import *
import time
import torch
from torch import nn
import numpy as np
import pickle
from torch.cuda.amp import autocast,GradScaler
import subprocess
import gc
import wandb
from traintest import train, validate
import dataloader_custom

In [2]:
from models import ASTModel

In [3]:
from utilities import *

In [4]:
mean = [-6.442367, 4.1776743]

In [5]:
label_csv = '../egs/custom/data/custom_labels.csv'
data_train = '../egs/custom/data/datafiles/custom_train_data_1.json'
data_eval = '../egs/custom/data/datafiles/custom_eval_data_1.json'

audio_conf = {'num_mel_bins': 128, 'target_length': 512, 'freqm': 10, 'timem': 10, 'mixup': 0, 
              'dataset': 'custom', 'mode':'train', 'mean':mean[0], 
              'std':mean[1],'noise':False}
val_audio_conf = {'num_mel_bins': 128, 'target_length': 512, 
                  'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': 'custom', 'mode':'evaluation', 
                  'mean':mean[0], 'std':mean[1], 'noise':False}

In [6]:
batch_size = 48

In [7]:
train_loader = torch.utils.data.DataLoader(
            dataloader_custom.AudiosetDataset(data_train, label_csv=label_csv, audio_conf=audio_conf),
            batch_size=48, shuffle=False, num_workers=0, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
        dataloader_custom.AudiosetDataset(data_eval, label_csv=label_csv, audio_conf=val_audio_conf),
        batch_size=batch_size*2, shuffle=False, num_workers=0, pin_memory=True)

audio_model = ASTModel(label_dim=21, fstride=10, tstride=10, input_fdim=128,
                                  input_tdim=512, imagenet_pretrain=True,
                                  audioset_pretrain=False, model_size='base384')

---------------the train dataloader---------------
now using following mask: 10 freq, 10 time
now using mix-up with rate 0.000000
now process custom
use dataset mean -6.442 and std 4.178 to normalize the input.
number of classes is 21
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process custom
use dataset mean -6.442 and std 4.178 to normalize the input.
number of classes is 21
---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=600


/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9/site-packages/torch/nn/functional.py:3454: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [11]:
audio_model = ASTModel(label_dim=21, fstride=10, tstride=10, input_fdim=128,
                                  input_tdim=512, imagenet_pretrain=True,
                                  audioset_pretrain=True, model_size='base384')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not isinstance(audio_model, nn.DataParallel):
    audio_model = nn.DataParallel(audio_model)
audio_model = audio_model.to(device)

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=600


In [12]:
audio_input, labels = next(iter(train_loader)) #now takes all data frome the first class => a000, lesred1

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_grad_enabled(True)
trainables = [p for p in audio_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(trainables, 1e-5, weight_decay=5e-7, betas=(0.95, 0.999))
loss_fn = nn.BCEWithLogitsLoss(reduction = 'none')#reduction = 'none'
warmup = False

In [18]:
torch.cuda.empty_cache()

In [19]:
audio_model.train()
scaler = GradScaler()
for i, (audio_input, labels) in enumerate(train_loader):
    print(audio_input.shape)
    print(i)        
    B = audio_input.size(0)
    audio_input = audio_input.to(device, non_blocking=True)
    labels = labels.to(device, non_blocking=True)

    with autocast():
            audio_output = audio_model(audio_input)
            
            predictions = audio_output.to('cpu').detach()
            print("audio_output is {out}\n".format(out = predictions))
            loss = loss_fn(audio_output, labels)
    print(loss)               
    loss = loss.mean()
    print('LOSS is {loss}'.format(loss = loss))
            
            # optimization if amp is not used
            # optimizer.zero_grad()
            # loss.backward()
            # optimizer.step()

            # optimiztion if amp is used
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scler.update()

stats, valid_loss = validate(audio_model, test_loader, args, epoch)

f1 = stats[0]
avg_precision = stats[1]
avg_recall = stats[2]

print("f1_score: {:.6f}".format(f1))
print("avg_precision_score: {:.6f}".format(avg_precision))
print("avg_recall_score: {:.6f}".format(avg_recall))
print("train_loss: {:.6f}".format(loss_meter.avg))

if f1 > best_f1:
    best_f1 = f1
if main_metrics == 'mAP':
    best_epoch = epoch

scheduler.step()

print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

epoch += 1

torch.Size([48, 512, 128])
0


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9/site-packages/torch/cuda/amp/autocast_mode.py", line 139, in decorate_autocast
    return func(*args, **kwargs)
  File "/scratch/users/fsofian19/COMP491_model/models/ast_custom/src/models/ast_models.py", line 177, in forward
    x = blk(x)
  File "/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/kuacc/users/fsofian19/.conda/envs/ast_env/lib/python3.9

In [5]:
def get_fbank(filename):
        # mixup

        waveform, sr = torchaudio.load(filename)
        waveform = waveform - waveform.mean()


        fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                  window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10)

        target_length = 512
        n_frames = fbank.shape[0]

        p = target_length - n_frames

        # cut and pad
        if p > 0:
            m = torch.nn.ZeroPad2d((0, 0, 0, p))
            fbank = m(fbank)
        elif p < 0:
            fbank = fbank[0:target_length, :]


        return fbank

In [34]:
cur_fbank = get_fbank('/kuacc/users/bbiner21/birdclef-2022/birdclef-2022-16khz/skylar/XC362045.ogg')

#/kuacc/users/bbiner21/birdclef-2022/birdclef-2022-16khz/akiapo/XC122401.ogg
#/kuacc/users/bbiner21/ast/egs/custom/data/custom_data/audio_16k/XC109605.ogg
# plt.figure()
# cur_fbank = torch.flip(cur_fbank.t(), [0])
# plt.imshow(cur_fbank.numpy(), cmap='gray',aspect='auto')
print(cur_fbank.shape)

norm_mean = -6.0138397
norm_std = 4.589279

cur_fbank = (cur_fbank - norm_mean) / (norm_std * 2)

cur_fbank = cur_fbank.unsqueeze(0)
out = audio_model(cur_fbank)
out = torch.sigmoid(out)
out

torch.Size([512, 128])


tensor([[0.3119, 0.1915, 0.2662, 0.1010, 0.2327, 0.4213, 0.1695, 0.3143, 0.1613,
         0.1568, 0.3740, 0.3630, 0.2916, 0.4650, 0.3148, 0.3887, 0.2310, 0.2693,
         0.7696, 0.2022, 0.2180]], grad_fn=<SigmoidBackward>)

In [37]:
a = calculate_stats(out.detach(),temp)

1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
